In [1]:
!pip install -q langchain sentence-transformers chromadb langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.8 MB/s eta 0:00:0

In [2]:
import os, re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from typing import List
import shutil
from tqdm import tqdm
import chromadb

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Ruta a la carpeta donde están los .txt
BASE_PATH = "/content/drive/MyDrive/Facultad/7mo Cuatrimestre - Marzo 2025/NLP/Tp Final NLP/datos/información/"
LIMPIOS_PATH = os.path.join(BASE_PATH, "txt_limpios")
os.makedirs(LIMPIOS_PATH, exist_ok=True)

# Configuración splitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " "],
    chunk_size=512,
    chunk_overlap=100,
    length_function=len
)

# Embeddings multilingües optimizados
EMBEDDING_MODEL_NAME = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
model_emb = SentenceTransformer(EMBEDDING_MODEL_NAME)

# Carpeta para persistencia de Chroma
CHROMA_PATH = "/content/drive/MyDrive/Facultad/7mo Cuatrimestre - Marzo 2025/NLP/Tp Final NLP/chroma_vectorial"
COLLECTION_NAME = "cascadia_chroma"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Limpieza de texto

In [4]:
keywords_to_remove = [
    "log in", "home", "contact", "forum", "wishlist", "cart", "instagram", "twitter", "facebook",
    "subscribe", "newsletter", "buy now", "kickstarter", "https://", "www.", "share", "review",
    "menu", "events", "about", "terms", "policy", "sitemap", "board game", "game", "shop", "podcast",
    "store", "shipping", "payment", "cookie", "login", "register"
]

def clean_text(text: str) -> str:
    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        line = line.strip()
        if not line or any(kw in line.lower() for kw in keywords_to_remove):
            continue
        if len(line.split()) <= 2 and not line.endswith("."):
            continue
        line = re.sub(r'[^\x00-\x7F]+', '', line)
        cleaned_lines.append(line)
    return '\n'.join(cleaned_lines)

# Procesar archivos .txt individuales y guardar limpios
archivos_txt = [f for f in os.listdir(BASE_PATH) if f.endswith(".txt")]

for archivo in archivos_txt:
    input_path = os.path.join(BASE_PATH, archivo)
    output_path = os.path.join(LIMPIOS_PATH, archivo)

    with open(input_path, 'r', encoding='utf-8') as f:
        raw_text = f.read()

    cleaned_text = clean_text(raw_text)

    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(cleaned_text)

    print(f"✅ {archivo} limpiado y guardado en txt_limpios")

✅ cascadia_manual.txt limpiado y guardado en txt_limpios
✅ whatboardgame_review.txt limpiado y guardado en txt_limpios
✅ oneboardfamily_review.txt limpiado y guardado en txt_limpios
✅ flatout_games.txt limpiado y guardado en txt_limpios
✅ boardgamereview_review.txt limpiado y guardado en txt_limpios
✅ board_game_co_uk_guide.txt limpiado y guardado en txt_limpios
✅ bluehighwaygames_description.txt limpiado y guardado en txt_limpios
✅ foro_reviews_cascadia.txt limpiado y guardado en txt_limpios
✅ cascadia_manual_limpio.txt limpiado y guardado en txt_limpios


# CHUNKS Y EMBEDDINGS

In [5]:
from tqdm import tqdm

vector_db_data = []
id_counter = 0

archivos_limpios = [f for f in os.listdir(LIMPIOS_PATH) if f.endswith(".txt")]

for archivo in tqdm(archivos_limpios):
    with open(os.path.join(LIMPIOS_PATH, archivo), "r", encoding="utf-8") as f:
        text = f.read()

    chunks = text_splitter.split_text(text)
    embeddings = model_emb.encode(chunks, show_progress_bar=False)

    for i, (chunk, vector) in enumerate(zip(chunks, embeddings)):
        vector_db_data.append({
            "id": str(id_counter),
            "content": chunk,
            "embedding": vector.tolist(),
            "metadata": {
                "file_name": archivo,
                "chunk_id": i,
                "source": archivo
            }
        })
        id_counter += 1

100%|██████████| 9/9 [02:57<00:00, 19.72s/it]


# CREAR Y GUARDAR LA BASE CHROMA

In [7]:
client = chromadb.PersistentClient(path=CHROMA_PATH)
collection = client.get_or_create_collection(name=COLLECTION_NAME)

In [8]:
# Agregar documentos a Chroma
collection.add(
    ids=[item["id"] for item in vector_db_data],
    embeddings=[item["embedding"] for item in vector_db_data],
    documents=[item["content"] for item in vector_db_data],
    metadatas=[item["metadata"] for item in vector_db_data]
)

print(f"Datos insertados en la colección '{COLLECTION_NAME}'")
print("Total de registros:", collection.count())

Datos insertados en la colección 'cascadia_chroma'
Total de registros: 498


In [10]:
result = collection.get(limit=3)

print("\n📄 Ejemplo de un documento almacenado:")
print("ID:", result['ids'][0])
print("Documento:", result['documents'][0])
print("Metadatos:", result['metadatas'][0])


📄 Ejemplo de un documento almacenado:
ID: 0
Documento: C A S C A D I A
RANDY FLYNN
of the Pacific Northwest. Designed by Randy Flynn, for 1-4 players, ages 10+.
The Pacific Northwest, also referred to as Cascadia (kas-KAE-Dee-uh), is a geographic region in
western North America bounded by the Pacific Ocean to the west and (loosely) by the Rocky
Mountains to the east. Though no official boundary exists, the most common conception
includes the Canadian province of British Columbia and Yukon Territory as well as the U.S.
states of Idaho, Oregon, and Washington.
Metadatos: {'source': 'cascadia_manual.txt', 'file_name': 'cascadia_manual.txt', 'chunk_id': 0}


# CONSULTAS A LA BBDD

In [11]:
clientChroma = chromadb.PersistentClient(path=CHROMA_PATH)
collection = client.get_or_create_collection(name=COLLECTION_NAME)

In [8]:
model_emb = SentenceTransformer(EMBEDDING_MODEL_NAME)

In [12]:
# Preguntas mixtas en inglés y español
test_queries = [
    "¿Cómo se gana el juego?",
    "How many rounds are there?",
    "¿Qué pasa al final de la partida?",
    "What are nature tokens used for?",
    "¿Cuántos puntos valen los osos?"
]

# Ejecutar consultas y mostrar resultados
for q in test_queries:
    print("="*60)
    print(f"🔎 Consulta: {q}")

    embedding = model_emb.encode([q])[0]
    results = collection.query(
        query_embeddings=[embedding],
        n_results=3
    )

    for i in range(len(results["documents"][0])):
        print(f"\n📎 Resultado #{i+1}")
        print("📝 Fragmento:", results["documents"][0][i][:300])
        print("📁 Archivo:", results["metadatas"][0][i]["file_name"])

🔎 Consulta: ¿Cómo se gana el juego?

📎 Resultado #1
📝 Fragmento: Kun pelaaja on asettanut laatan ja elinmerkin maisemaansa, knnetn varannosta uusi laatta ja elinmerkki valittujen tilalle ja seuraava pelaaja aloittaa vuoronsa. Pelaajamrst riippumatta jokaisella pelaajalla on aina peliss 20 vuoroa aikaa rakentaa omaa maisemaansa ja kerrytt potentiaalista pistesaldo
📁 Archivo: foro_reviews_cascadia.txt

📎 Resultado #2
📝 Fragmento: Lisksi tuuria tasoitetaan pelin aikana ansaittavilla kvyill (peliss nit kutsutaan luontomerkeiksi). Pelaaja voi halutessaan kytt vuorollaan kpyjn, jolloin hn voi valita mink tahansa tarjolla olevista laatoista ja elinmerkeist - niiden ei siis tarvitse olla valmis pari, kuten normaalisti pitisi. Vaih
📁 Archivo: foro_reviews_cascadia.txt

📎 Resultado #3
📝 Fragmento: Alkuvalmistelut tekee ripesti. Pelaajille jaetaan aloituslaatat, elinympristlaattoja otetaan pelaajamrn mukaisesti varantoon ja jokaiselle elinlajille valitaan pisteytyskortti. Lopuksi nelj elinymprist